In [ ]:
########################################################################
#
# Use 2D Bragg ptycho code to generate 3D simulated CDI from illuminated vortex structure
# Author: Vincent Favre-Nicolin <favre@esrf.fr>
#
########################################################################
import os
import timeit
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from ipywidgets import interact, IntSlider

# Use only OpenCL - this must be done before PyNX imports
os.environ['PYNX_PU']='opencl.p6000'  # Force using OpenCL only (even for Wavefront)

from pynx.ptycho.bragg2d import *
from pynx.ptycho.bragg.cpu_operator import show_3d
from pynx.ptycho.simulation import spiral_archimedes
from pynx.wavefront import Wavefront, ImshowRGBA, ImshowAbs, PropagateFarField, PropagateNearField
from pynx.utils.plot_utils import complex2rgbalin


In [ ]:
# Experiment parameters
wavelength = 1.5e-10
delta = np.deg2rad(64)
eta = delta/2
nu = np.deg2rad(0)
pixel_size_detector = 55e-6
ny, nx = (400, 400)
detector_distance = 1

# Spiralscan positions - we only need one for this simulation
nb = 1

xs, ys = spiral_archimedes(80e-9, nb)
zs = np.zeros_like(xs)

# Rotate positions according to eta (piezo motors in the sample frame are rotated by eta)
ce, se = np.cos(eta), np.sin(eta)
ys, zs = ce * ys + se * zs, ce * zs - se * ys

# Project the positions along z onto the average sample position, to avoid non-centered Psi
zs += ys / np.tan(eta)

# detector parameters
detector = {'rotation_axes': (('x', delta), ('y', nu)), 'pixel_size': pixel_size_detector,
            'distance': detector_distance}
plt.figure()
plt.plot(xs*1e6, ys*1e6, '.')

In [ ]:
# This sets the number of frames which are handled simultaneously during processing.
# Larger numbers (up to the total number of frames) are more efficient, but more memory-hungry
default_processing_unit.cl_stack_size = 1 # nb


In [ ]:
# Create empty data
data = Bragg2DPtychoData(iobs=np.empty((nb, ny, nx), dtype=np.float32), positions=(xs, ys, zs), mask=None,
                       wavelength=wavelength, detector=detector)

In [ ]:
if False:
    # Import existing probe from 2D ptycho
    d = np.load("/Users/favre/Analyse/201606id01-FZP/ResultsScan0013/latest.npz")
    #d = np.load("/Users/vincent/Analyse/201606id01-FZP/ResultsScan0000/latest.npz")
    pr = Wavefront(d=np.fft.fftshift(d['probe'],axes=(-2,-1)), z=0, pixel_size=d['pixelsize'], wavelength=wavelength)
else:
    # Simulate probe from a focused circular aperture, with large defocusing to get a wide beam
    pixel_size_focus = wavelength * detector_distance / (nx * pixel_size_detector)
    focal_length = 0.09
    defocus = 200e-6
    radius = 200e-6
    pixel_size_aperture = wavelength * focal_length / (nx * pixel_size_focus)
    pr = Wavefront(d=np.ones((ny, nx)), wavelength=wavelength, pixel_size=pixel_size_aperture)
    x, y = pr.get_x_y()
    r = np.sqrt(x**2+y**2)
    print(x.min(),x.max(), y.min(), y.max(), wavelength, pixel_size_aperture)
    pr.set(r < radius)
    pr = PropagateNearField(dz=defocus) * PropagateFarField(focal_length, forward=False) * pr

print('Probe pixel size: %6.2fnm'%(pr.pixel_size*1e9))
pr = ImshowRGBA()*pr

In [ ]:
# Create main Bragg Ptycho object
p = Bragg2DPtycho(probe=pr, data=data, support=None)
pxyz = p.voxel_size_object()
print(wavelength * detector_distance / (pixel_size_detector*nx)*1e9)
print("Object voxel size: %6.2fnm x %6.2fnm x %6.2fnm" % (pxyz[0] * 1e9, pxyz[1] * 1e9, pxyz[2] * 1e9))
print(p.m)

In [ ]:
# Base parallelepiped object
x0, x1, y0, y1, z0, z1 = -1e-6, 1e-6, -200e-9, 200e-9, -1e-6, 1e-6
# Create a support. Larger than the object, or not...
rs = 1.0
# Equation for GPU init of support using Monte-Carlo integration
eq = "(x >= %g) * (x <= %g) * (y >= %g) * (y <= %g) * (z >= %g) * (z <= %g)" % (rs * x0, rs * x1, rs * y0, rs * y1, rs * z0, rs * z1)
print(eq)
s = InitSupport(eq, rotation_axes=[('x', eta)], shrink_object_around_support=True)
p = s * p
print(p.support.sum()/p.support.size)
if True:
    #plt.figure(figsize=(9,4))
    #show_3d(p.support,ortho_m=p.m, rotation=('x', -eta))
    plt.figure(figsize=(9,4))
    show_3d(p.support,ortho_m=p.m, rotation=None, title='Support-rotation=None')
    plt.figure(figsize=(9,4))
    show_3d(p.support,ortho_m=p.m, rotation=('x', eta), title='Support-rotated back // to xz plane')


In [ ]:
obj0 = p.support / 100
obj1 = obj0.copy()

x,y,z = p.get_xyz(domain='object', rotation=('x', -eta))
# Add some strain
obj1 = obj0 * np.exp(1j * np.arctan2(z,x) * np.exp(-(x ** 2 + z ** 2) / 0.5e-6 ** 2))

p.set_obj(obj1)
plt.figure(figsize=(9,4))
p = ShowObj(rotation=('x', eta), title='Object (eta=%6.2f°, delta=%6.2f°)' % (np.rad2deg(eta), np.rad2deg(delta))) * p
#p = ShowObj(rotation=('x', eta), title='Object rotated back to eta=0') * p
print(x0*1e6,x1*1e6,y0*1e6,y1*1e6,z0*1e6,z1*1e6)

In [ ]:
# Compute the pixel coordinate of the center of the object*probe for each frame
p = CalcCenterObjProbe() * p
#print(p._cl_obs_v[0].cl_cixo, p._cl_obs_v[0].cl_ciyo)

In [ ]:
# This will calculate the Object*Probe 3D array
p = ObjProbe2PsiDebug(npsi=1, calc_3d=True) * p

# Display calculated Psi before propagation to the detector
plt.figure(figsize=(9,5))
show_3d(p._psi3d[0,0,0],ortho_m=p.m, rotation=('x', eta), title='Obj*Probe')

In [ ]:
# Compute the 3D CDI diffracted intensity from the Obj*probe object, and save it
icalc_cdi = abs(np.fft.fftshift(np.fft.fftn(p._psi3d[0,0,0])))**2

# Fix total number of photons and apply poisson noise
icalc_cdi = np.random.poisson(icalc_cdi * 1e9 / icalc_cdi.sum())
print(icalc_cdi.shape)
np.savez_compressed('cdi_vortex_simulated.npz', iobs= icalc_cdi)

In [ ]:
# Look at the simulated data:
plt.figure(figsize=(9,4))
vmax = icalc_cdi.max()
def plot_obs_calc(i):
    plt.clf()
    plt.imshow(icalc_cdi[i],origin='lower', norm=LogNorm(vmin=1, vmax=vmax))
    plt.colorbar()

interact(plot_obs_calc, i=IntSlider(min=0,max=len(icalc_cdi)-1,step=1,value=len(icalc_cdi)//2))

In [ ]:
# Solve this CDI from the command line using e.g.:
#pynx-id01cdi.py data=cdi_vortex_simulated.npz support_threshold_method=max support_threshold=0.05,0.12 nb_run=20 nb_run_keep=4
